# Automatically generate FOVs given a non-TMA or TMA

This automatically creates tiles which define FOVs on the MIBI slide based on user-set parameters.

Notes about MIBI coordinate axes:

* The x-axis goes from left to right ascending
* The y-axis goes from bottom to top ascending

In [ ]:
import json
import os

from ark.mibi import tiling_utils

### Define paths to the JSON data

Define the following parameters to set your home directory:

* `base_dir`: the root directory of the tiling data
* `json_tiling_dir`: the directory in `base_dir` containing the information to read and write the FOV info

Define the prefix to use for your input and output file names:

* `fov_gen_prefix`: defaults to `example_tma`

Define the following input and output paths (all prefixed by `fov_gen_prefix`):

* `fov_list_path`: the list of FOVs to use for tiling
* `fov_output_path`: where to store the FOVs. Will be an actual run file for non-TMAs and a basic mapping of FOV to centroid coordinate (in microns) for TMAs.
* `moly_path`: the path to the Moly point, needed if you want to insert this between FOVs and/or regions after remapping. Ignored if you choose not to insert Moly points between FOVs or if you're tiling TMAs.

More about `fov_list_path`:

* non-TMAs: each FOV in `fov_list_path` defines the top-left corner of a separate region
* TMAs: `fov_list_path` should contain 2 FOVs, one defining the upper-left corner and one defining the bottom-right corner FOV

In [ ]:
# define your home directory
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
fov_gen_prefix = 'example_tma'

# define the input and output files
fov_list_path = os.path.join(json_tiling_dir, '%s_fov_list.json' % fov_gen_prefix)
fov_output_path = os.path.join(json_tiling_dir, '%s_fov_output.json' % fov_gen_prefix)
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % fov_gen_prefix)

### Define whether you're tiling a TMA or not

In [ ]:
tma = True

### Set tiling parameters (non-TMAs)

<b>NOTE: ignore this section if you're doing TMA tiling</b>

For non-TMAs, the following parameters need to be defined for each region (you will be prompted for them running this cell):

* The FOV defining the top-left corner and its centroid (taken from `fov_list_path`, each entry indicates a different region)
* Number of FOVs along the x- and y-axis (set by you)
* x- and y-axis step size (set by you)
* Whether to randomize the order of the FOVs or not (set by you)
* Whether to insert Moly points between regions
* Whether to insert Moly points between FOVs at a specified interval. Please refer to the documentation for `generate_fov_list_non_tma` for how the Moly point insertion works: https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.mibi.html#ark.mibi.tiling_utils.generate_fov_list_non_tma

In [ ]:
if not tma:
    tiling_params, moly_point = tiling_utils.set_tiling_params_non_tma(
        fov_list_path,
        moly_path
    )

### Set tiling parameters (TMAs)

<b>NOTE: ignore this section if you're doing non-TMA tiling</b>

For TMAs, define the following:

* `tma_num_x`: the number of FOVs along the x-axis to create. Note that you must define at least 3.
* `tma_num_y`: the number of FOVs along the y-axis to create. Note that you must define at least 3.

The FOVs created will be equally spaced along the coordinate grid defined by the top-left and bottom-right FOV in `fov_list_path`.

NOTE: for TMAs, randomization and Moly point insertion will be done in `example_remap_tma_fov.ipynb` after the names have been remapped.

In [ ]:
tma_num_x = 4
tma_num_y = 7

### Create the FOV tiles for each region

Tiling will be done from top to bottom (descending y-axis order), then left to right (ascending x-axis order). FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed.

For example, `R1C1` means the FOV is in row 1 and column 1 of the grid of FOVs, `R2C1` would be the next FOV down, and `R1C2` would be the next FOV to the right. In an unrandomized `auto_fov_regions`, `R2C1` would immediately follow `R1C1` and all of the `R{n}C1` FOVs would come before the `R{n}C2` FOVs for each region.

Note that due to the y-axis ascending from bottom to top, `R2C1's` y-axis coordinate is lower compared to `R1C1's`.

In [ ]:
# generate the list of FOVs
if tma:
    auto_fov_regions = tiling_utils.generate_fov_list_tma(
        fov_list_path,
        tma_num_x,
        tma_num_y
    )
else:
    auto_fov_regions = tiling_utils.generate_fov_list_non_tma(
        tiling_params,
        moly_point
    )

In [ ]:
# write the tiled output
with open(fov_output_path, 'w') as fop:
    json.dump(auto_fov_regions, fop)